In [17]:
import pandas as pd
import numpy as np
import copy as cp

import statsmodels.api as sm
import sklearn.metrics as metrics

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
onedrive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"

In [3]:
repeat_rate = pd.read_csv(onedrive_path + "Data/Repeat Rate/combined.csv")
ratings = pd.read_csv(onedrive_path + "Data/Ratings and Reviews/reviews_demand_itemid4.csv")
cc = pd.read_csv(onedrive_path + "Data/Consumer Care/cc_product_month.csv")

In [4]:
cc = cc.rename(columns = {"P4": "itemid_4",
                         "Date Month": "date"})

## Summarize Ratings at FY Level

In [5]:
#ratings = ratings.merge(cc, how = "left")

In [6]:
ratings.date = pd.to_datetime(ratings.date)

In [7]:
FY18_MIN = pd.to_datetime("7/1/2018")
FY19_MIN = pd.to_datetime("7/1/2019")
FY19_MAX = pd.to_datetime("7/1/2020")

ratings["fy"] = [int(18) if x >= FY18_MIN and x < FY19_MIN else int(19) if x >= FY19_MIN and x < FY19_MAX else 0 for x in ratings.date]

In [8]:
ratings = ratings.groupby(["itemid_4", "fy"]).sum().reset_index()

In [9]:
repeat_rate.loc[repeat_rate.brand == "Jo Malone London"]


,fy,brand,itemid_4,category,subcategory,description,transaction_count,percent_brand_mix,product_repeat,brand_repeat,purchase_cycle_days,retail_price,retail_transactions,online_transactions,percent_retail_transactions,percent_online_transactions
321,18,Jo Malone London,6902,FRAGRANCE,WILD BLUEBELL,WILD BLUEBELL COLOGNE 30ML/SLI,8263,0.037636,0.197964,0.364065,82.753508,65.005836,624,7639,0.075517,0.924483
322,18,Jo Malone London,6902,FRAGRANCE,WOOD SAGE AND SEA SALT,WOOD SAGE AND SEA SALT 30ML,7443,0.033901,0.077228,0.268563,93.365063,65.022058,1386,6057,0.186215,0.813785
323,18,Jo Malone London,6902,FRAGRANCE,ENGLISH PEAR & FREESIA,ENGLISH PEAR & FREESIA CLGNE 3,6846,0.031182,0.088330,0.296050,86.046518,65.003569,1160,5686,0.169442,0.830558
324,18,Jo Malone London,6902,SKINCARE,VITAMIN E LIP CONDITIONER,VITAMIN E LIP CONDITIONER,5278,0.024040,0.193330,0.422838,55.167446,29.998124,342,4936,0.064797,0.935203
325,18,Jo Malone London,6902,FRAGRANCE,SOAP,EP&F SOAP,5047,0.022988,0.208962,0.490683,74.028390,21.899528,341,4706,0.067565,0.932435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,19,Jo Malone London,6900,FRAGRANCE,ENGLISH PEAR & FREESIA SET,ENGLISHPEAR&FREESIASET,1,0.000004,0.000000,0.000000,0.000000,135.000000,1,0,1.000000,0.000000
2623,19,Jo Malone London,6900,FRAGRANCE,BODY & HAND WASH,P&BS BODY & HAND WASH/NEW ILN,1,0.000004,0.000000,1.000000,0.000000,40.000000,0,1,0.000000,1.000000
2624,19,Jo Malone London,6900,FRAGRANCE,BODY & HAND WASH,OB BODY & HAND WASH 250ML,1,0.000004,0.000000,0.000000,0.000000,40.000000,1,0,1.000000,0.000000
2625,19,Jo Malone London,6900,FRAGRANCE,BATH OIL,POMEGRANATE NOIR BATH OIL,1,0.000004,0.000000,0.000000,0.000000,65.000000,1,0,1.000000,0.000000


## Join with Repeat Rate

In [10]:
merged = repeat_rate.merge(ratings, on =["itemid_4", "fy"])

In [11]:
merged = merged.loc[merged.demand > 0]
merged = merged.loc[merged.nb_reviews > 10]

In [12]:
merged.columns

Index(['fy', 'brand', 'itemid_4', 'category', 'subcategory', 'description',
       'transaction_count', 'percent_brand_mix', 'product_repeat',
       'brand_repeat', 'purchase_cycle_days', 'retail_price',
       'retail_transactions', 'online_transactions',
       'percent_retail_transactions', 'percent_online_transactions',
       'avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand',
       'demand_1', 'demand_2', 'demand_3'],
      dtype='object')

## Predict Repeat Rates

### Brand Repeat Rate

In [25]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand']]
y = merged["brand_repeat"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [27]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-9.902418370804611

In [28]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                 -0.028
Method:                 Least Squares   F-statistic:                    0.6612
Date:                Wed, 22 Jul 2020   Prob (F-statistic):              0.773
Time:                        11:59:31   Log-Likelihood:                 137.79
No. Observations:                 136   AIC:                            -251.6
Df Residuals:                     124   BIC:                            -216.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3735      0

### Product Repeat Rate

In [29]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand']]
y = merged["product_repeat"]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [31]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-1.778936014552512

In [32]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     3.178
Date:                Wed, 22 Jul 2020   Prob (F-statistic):           0.000796
Time:                        12:01:24   Log-Likelihood:                 225.24
No. Observations:                 136   AIC:                            -426.5
Df Residuals:                     124   BIC:                            -391.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0747      0

## Try removing the demand variable
### Brand Repeat

In [33]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment']]
y = merged["brand_repeat"]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [35]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-29.452237884403402

In [36]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.161
Date:                Wed, 22 Jul 2020   Prob (F-statistic):              0.323
Time:                        12:06:50   Log-Likelihood:                 171.79
No. Observations:                 136   AIC:                            -321.6
Df Residuals:                     125   BIC:                            -289.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3763      0

Average number of statements is showing significance which is interesting as we know that that is corellatied with negative reviews

### Product Repeat

In [37]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment']]
y = merged["product_repeat"]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [39]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-2.292887699010508

In [40]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1.585
Date:                Wed, 22 Jul 2020   Prob (F-statistic):              0.119
Time:                        12:08:34   Log-Likelihood:                 213.51
No. Observations:                 136   AIC:                            -405.0
Df Residuals:                     125   BIC:                            -373.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0815      0

For the product repeat rating three is starting to show something. Let's look at sentiment because there's a crazy amount of multicolinearity going on here

## Sentiment
### Brand Repeat Rate

In [41]:
X = merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']]
y = merged["brand_repeat"]

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [43]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-66.20959553670018

In [44]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7643
Date:                Wed, 22 Jul 2020   Prob (F-statistic):              0.550
Time:                        12:10:58   Log-Likelihood:                 131.48
No. Observations:                 136   AIC:                            -253.0
Df Residuals:                     131   BIC:                            -238.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.4084      0

Nada

### Product Repeat Rate

In [53]:
X = merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']]
y = merged["product_repeat"]

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [55]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-20.68099506199587

In [56]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     3.123
Date:                Wed, 22 Jul 2020   Prob (F-statistic):             0.0172
Time:                        12:12:31   Log-Likelihood:                 211.02
No. Observations:                 136   AIC:                            -412.0
Df Residuals:                     131   BIC:                            -397.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1008      0

Some small effect from sentiment negative, but the sign is wrong. Let's try logging the right hand side of the regression

## Logged Features
### Brand Repeat

In [57]:
X = np.log(merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']] + 1)
y = merged["brand_repeat"]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [59]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-218.6447877874172

In [60]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                    0.1263
Date:                Wed, 22 Jul 2020   Prob (F-statistic):              0.973
Time:                        12:14:08   Log-Likelihood:                 123.96
No. Observations:                 136   AIC:                            -237.9
Df Residuals:                     131   BIC:                            -223.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3967      0

Ummmm...nope

### Product Repeat

In [61]:
X = np.log(merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']] + 1)
y = merged["product_repeat"]

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [63]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-23.606669195105127

In [64]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     2.906
Date:                Wed, 22 Jul 2020   Prob (F-statistic):             0.0242
Time:                        12:14:55   Log-Likelihood:                 217.04
No. Observations:                 136   AIC:                            -424.1
Df Residuals:                     131   BIC:                            -409.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1032      0

Sentiemnt negative keeps on having a positive coefficent, but I'm not sure what to think of it...the R^2 is crazy low, so I don't want to put stock in that